<a href="https://colab.research.google.com/github/1kaiser/test2022/blob/main/imd_NetCFD_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Convert NetCDF file(s) to CSV file(s)**

In [ ]:
# Replace 'local_storage_directory', 'netcdf_dir' and 'csv_dir' by respectively
# the directory path to Copernicus Marine data, the directory path to netcdf files
# and the directory path to csv files
local_storage_directory = '/content/'
netcdf_dir = local_storage_directory + 'netcdf/'
!mkdir -p {netcdf_dir}
csv_dir = local_storage_directory + 'csv/'
!mkdir -p {csv_dir}
!python -m pip install xarray

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
import xarray as xr
import os

# Set a new directory path where the list of netcdf files (.nc) is stored.
# Path should end with a 'slash (/)'
files_to_convert = local_storage_directory + 'netcdf/'
# Set a loop to create a .csv file for each .nc file listed in `files_to_convert`
for netcdf_file_name in os.listdir(files_to_convert):
    ds = xr.open_dataset(files_to_convert + netcdf_file_name)
    df = ds.to_dataframe()
    df.to_csv(csv_dir + netcdf_file_name[:-3] + '.csv')
    print (netcdf_file_name + ' has been processed to .csv')
    print (df)






PBLH202104_V1.nc has been processed to .csv
         latitude  longitude    pblh
lt  ln                              
0   0       5.125     50.125 -9999.0
    1       5.125     50.375 -9999.0
    2       5.125     50.625 -9999.0
    3       5.125     50.875 -9999.0
    4       5.125     51.125 -9999.0
...           ...        ...     ...
139 235    39.875    108.875 -9999.0
    236    39.875    109.125 -9999.0
    237    39.875    109.375 -9999.0
    238    39.875    109.625 -9999.0
    239    39.875    109.875 -9999.0

[33600 rows x 3 columns]
_Clim_Pred_LRF_New_RF25_IMD0p252021.nc has been processed to .csv
                               RAINFALL
LONGITUDE LATITUDE TIME                
66.5      6.5      2021-01-01       NaN
                   2021-01-02       NaN
                   2021-01-03       NaN
                   2021-01-04       NaN
                   2021-01-05       NaN
...                                 ...
100.0     38.5     2021-12-27       NaN
                   2021

In [26]:
import pandas as pd
import os

for csv_file_name in os.listdir(csv_dir):
    csv_uncleaned_in = csv_file_name[:-4] + '.csv'
    csv_cleaned_out = csv_uncleaned_in[:-4] + '_cleaned.csv'
    data = pd.read_csv(csv_dir + csv_uncleaned_in)
    print(data.dropna())

    data.dropna().to_csv(csv_dir + csv_cleaned_out, index = False)


         LONGITUDE  LATITUDE        TIME  RAINFALL
307695       68.00     23.75  2021-01-01       0.0
307696       68.00     23.75  2021-01-02       0.0
307697       68.00     23.75  2021-01-03       0.0
307698       68.00     23.75  2021-01-04       0.0
307699       68.00     23.75  2021-01-05       0.0
...            ...       ...         ...       ...
5823570      97.25     28.25  2021-12-27       0.0
5823571      97.25     28.25  2021-12-28       0.0
5823572      97.25     28.25  2021-12-29       0.0
5823573      97.25     28.25  2021-12-30       0.0
5823574      97.25     28.25  2021-12-31       0.0

[1811860 rows x 4 columns]
         LONGITUDE  LATITUDE        TIME  RAINFALL
0            68.00     23.75  2021-01-01       0.0
1            68.00     23.75  2021-01-02       0.0
2            68.00     23.75  2021-01-03       0.0
3            68.00     23.75  2021-01-04       0.0
4            68.00     23.75  2021-01-05       0.0
...            ...       ...         ...       ...
181

In [27]:
!python -m pip install imdlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [30]:
import imdlib as imd
import numpy as np
import pandas as pd
""" 
# install imdlib python library
# you should be connected to internet for downloading the data
#-9999 value is for no data in saved csv file
# This code will download the imd data first and then convert the data to csv file
if you have data already downloaded then create folder named rain/tmax/tmin inside any folder and 
copy yearly data files in the respective folder and rename yearly data file as year name i.e 1951.GRD 1952.GRD etc and 
comment the line imd.get_data(variable,start_yr) and run the code it will convert the binary .GRD data into csv file
"""
start_yr = 2020 # give starting year from which you want to download/convert data: 1901 ownwards for rainfall, 1951 for tmax and tmin
end_yr = 2021 # give ending year upto which you want to download/convert data
variable = 'rain' # give variable name (rain for rainfall, tmax or tmin for min or max temperature)
file_format = 'yearwise' # other option (None), which will assume deafult imd naming convention
imd.get_data(variable, start_yr, end_yr, fn_format='yearwise', file_dir='/content/') # download IMD data: just change path as per your requirement
file_dir = '/content/' # this path should be same as mentioned in previous line
data = imd.open_data(variable, start_yr, end_yr,'yearwise', file_dir) # this will open the data downloaded and saved in the location mentioned in previous line
if variable == 'rain':
    grid_size = 0.25 # grid spacing in deg
    y_count = 129 # no of grids in y direction
    x_count = 135 # no of grids in x direction
    x = 66.5 # starting longitude taken from control file (.ctl)
    y = 6.5 # starting latitude taken from control file (.ctl)
elif variable == 'tmax' or variable == 'tmin':
    grid_size = 1 # grid spacing in deg
    y_count = 31 # no of grids in y direction
    x_count = 31 # no of grids in x direction
    x = 67.5 # starting longitude taken from control file (.ctl)
    y = 7.5 # starting latitude taken from control file (.ctl)

#print(grid_size,x_count, y_count, x, y)
data
data.shape
np_array = data.data
#print(np_array[0,0,0])
#xr_objecct = data.get_xarray()
#type(xr_objecct)
#xr_objecct.mean('time').plot()
years_no = (end_yr - start_yr) + 1
#print(years_no)
day = 0
for yr in range(0,years_no):
    f = open("/content/"+str(start_yr+yr)+"_"+str(variable)+".csv",'w') # just change the path where you want to save csv file
    if ((start_yr+yr) % 4 == 0) and ((start_yr+yr) % 100 != 0):  # check for leap year
        days = 366
        count = yr + days
    elif ((start_yr+yr) % 4 == 0) and ((start_yr+yr) % 100 == 0) and ((start_yr+yr) % 400 == 0):
        days = 366
        count = yr + days
    else:
        days = 365
        count = yr + days

    day = day + days

    f.write("X,Y,")
    for d in range(0, days):
        f.write(str(d+1))
        f.write(",")
    f.write("\n")
    #print(np_array[364,0,0])
    for j in range(0, y_count):

        for i in range(0, x_count):

            f.write(str((i * grid_size) + x))
            f.write(",")
            f.write(str((j * grid_size) + y))
            f.write(",")
            time = 0
            for k in range(day-days, day):

                val = np_array[k,i,j]
                if val == 99.9000015258789 or val == -999:
                    f.write(str(-9999))
                    f.write(",")
                else:
                    f.write(str(val))
                    f.write(",")


            f.write("\n")
    print("File for " + str(start_yr + yr) + "_" + str(variable) + " is saved")
print("CSV conversion successful !")

Downloading: rain for year 2020
Downloading: rain for year 2021
Download Successful !!!
(731, 135, 129)
File for 2020_rain is saved
File for 2021_rain is saved
CSV conversion successful !
